In [238]:
import datetime
import csv

In [239]:
import pymongo
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
#client = MongoClient('localhost', 27017)
#db=client.admin
# Issue the serverStatus command and print the results
#serverStatusResult=db.command("serverStatus")

In [240]:
#client = MongoClient('localhost',27017)

In [241]:
#db = client.CQT

In [242]:
#db.collection_names()

In [243]:
class parameter:
    def __init__(self,pname,idx=None,node=None):
        self.name = pname
        self.Ts = None
        self.Vals = None
        self.idx  = idx
        self.node = node
    def getdata(self,db,Ts_conditions, idx=None, node=None):
        conditions = []
        if self.name is None:
            print ("parameter missing in getdata")
            return
        
        conditions.append({'Param.Name':str(self.name)})
        if self.idx is not None:
            conditions.append({'Param.Index': int(self.idx)})
        if self.node is not None:
            conditions.append( {'Param.Node': int(self.node)})
        
        conditions.append({'Ts':Ts_conditions})
        results = db.ParamData.find( {'$and':conditions} ).sort('Ts',pymongo.DESCENDING)
        
        self.Ts = []
        self.Vals = []
        for x in results:   
            #[TODO] compare with the last value, if the jump is too big ignor, otherwise include. 
            #if x['Val'] == 0:
            #    continue 
            self.Ts.append(x['Ts'])
            self.Vals.append(x['Val'])
    
    def status (self):
        print (self.name, "idx=",self.idx,"node=",self.node, "len(Ts)=",len(self.Ts),"len(Vals)=", len(self.Vals))


In [244]:
class EPS:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}
        
        self.params['vbatt'] = parameter('vbatt')
        
        self.params['temp_0'] =  parameter('temp',idx=0)
        
        self.params['temp_1'] =  parameter('temp',idx=1)
        self.params['temp_2'] =  parameter('temp',idx=2)
        
        self.params['temp_3'] =  parameter('temp',idx=3)
        
        self.params['temp_4'] =  parameter('temp',idx=4)
        self.params['temp_5'] =  parameter('temp',idx=5)
        
        # Solar power input
        self.params['Vin_0'] = parameter('vboost',idx=0,node=2) #node = 2 by default here
        self.params['Vin_1'] = parameter('vboost',idx=1,node=2)
        self.params['Vin_2'] = parameter('vboost',idx=2,node=2)
        self.params['Cin_0'] = parameter('curin',idx=0,node=2) 
        self.params['Cin_1'] = parameter('curin',idx=1,node=2)
        self.params['Cin_2'] = parameter('curin',idx=2,node=2)     
        
        # Power output
        self.params['V_obc'] = parameter('out_val',idx=0,node=2) 
        self.params['V_radio'] = parameter('out_val',idx=3,node=2)
        self.params['V_payload'] = parameter('out_val',idx=5,node=2)
        self.params['C_obc'] = parameter('curout',idx=0,node=2) 
        self.params['C_radio'] = parameter('curout',idx=3,node=2)
        self.params['C_payload'] = parameter('curout',idx=5,node=2)
        
        self.params['battmode'] = parameter('battmode')
        #self.params['crusun'] = parameter('crusun')
        #self.params['crusys'] = parameter('crusys')

        # Watchdog timers and bootcauses
        self.params['wdt_i2c'] = parameter('wdtI2cS') 
        self.params['wdt_GND'] = parameter('wdtGndS')
        self.params['boot_cnt'] = parameter('cntBoot')        
        self.params['cntWdtI2c'] = parameter('cntWdtI2c') 
        self.params['cntwdtGND'] = parameter('cntWdtGnd')
        self.params['cntWdtCsp_0'] = parameter('cntWdtCsp',idx=0)         
        self.params['cntWdtCsp_1'] = parameter('cntWdtCsp',idx=1)         
        self.params['bootCause'] = parameter('bootcause')                 
        
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions)
            

In [245]:
class OBC:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}

        self.params['curGSSB1'] = parameter('curGSSB1')               
        self.params['curGSSB2'] = parameter('curGSSB2')
        self.params['curflash'] = parameter('curFlash')
        self.params['curPWM'] = parameter('curPWM')               
        self.params['temp_a'] = parameter('temp_a')  
        self.params['temp_b'] = parameter('temp_b')               
        self.params['pwrGSSB1'] = parameter('pwrGSSB1')               
        self.params['pwrGSSB2'] = parameter('pwrGSSB2')               
        self.params['pwrflash'] = parameter('pwrFlash')               
        self.params['pwrPWM'] = parameter('pwrPWM')               
        self.params['swload_count'] = parameter('swload_count')               
        self.params['fs_mounted'] = parameter('fs_mounted')               
        self.params['boot_count'] = parameter('boot_count')               
        self.params['boot_cause'] = parameter('boot_cause')               
        self.params['clock'] = parameter('clock')                             
        
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions)  

In [246]:
class AX100:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}
        
        self.params['temp_brd'] = parameter('temp_brd')
        self.params['temp_pa'] = parameter('temp_pa')
        self.params['last_rssi'] = parameter('last_rssi')
        self.params['last_rferr'] = parameter('last_rferr')
        self.params['bgnd_rssi'] = parameter('bgnd_rssi')
        self.params['tx_duty'] = parameter('tx_duty')
        self.params['tot_tx_cnt'] = parameter('tot_tx_count')
        self.params['tot_rx_cnt'] = parameter('tot_rx_count')
        self.params['tot_tx_bytes'] = parameter('tot_tx_bytes')
        self.params['tot_rx_bytes'] = parameter('tot_rx_bytes')
        self.params['boot_count'] = parameter('boot_count')
        self.params['boot_cause'] = parameter('boot_cause')
        self.params['tx_bytes'] = parameter('tx_bytes')
        self.params['rx_bytes'] = parameter('rx_bytes')
        self.params['active_conf'] = parameter('active_conf')
        self.params['tx_count'] = parameter('tx_count')
        self.params['rx_count'] = parameter('rx_count')
              
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions)  

In [247]:
class ADCS:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}
        
        self.params['gyro_0'] =  parameter('gyro',idx=0)
        self.params['gyro_1'] =  parameter('gyro',idx=1)
        self.params['gyro_2'] =  parameter('gyro',idx=2)
        self.params['gyro_trend_0'] =  parameter('gyro_trend',idx=0)
        self.params['gyro_trend_1'] =  parameter('gyro_trend',idx=1)
        self.params['gyro_trend_2'] =  parameter('gyro_trend',idx=2)
        self.params['gyro_temp'] = parameter('gyro_temp')
        self.params['gyro_valid'] = parameter('gyro_valid')
        
        self.params['mag_0'] =  parameter('mag',idx=0)
        self.params['mag_1'] =  parameter('mag',idx=1)
        self.params['mag_2'] =  parameter('mag',idx=2)
        self.params['mag_valid'] = parameter('mag_valid')
        
        self.params['suns_temp_0'] =  parameter('suns',idx=0)
        self.params['suns_temp_1'] =  parameter('suns',idx=1)
        self.params['suns_temp_2'] =  parameter('suns',idx=2)
        self.params['suns_temp_3'] =  parameter('suns',idx=3)
        self.params['suns_temp_4'] =  parameter('suns',idx=4)
        self.params['suns_temp_5'] =  parameter('suns',idx=5)  
        self.params['mag_valid'] =  parameter('mag_valid') 
        
        self.params['torquer_duty_0'] =  parameter('torquer_duty',idx=0)
        self.params['torquer_duty_1'] =  parameter('torquer_duty',idx=1)
        self.params['torquer_duty_2'] =  parameter('torquer_duty',idx=2)  
        
        self.params['status_mag'] = parameter('status_mag')
        self.params['status_css'] = parameter('status_css')
        self.params['status_gyro'] = parameter('status_gyro')
        self.params['status_bdot'] = parameter('status_bdot')
        self.params['status_run'] = parameter('status_run')
        self.params['looptime'] = parameter('looptime')
        self.params['maxlooptime'] = parameter('maxlooptime')
        self.params['bdot_rate_0'] = parameter('bdot_rate',idx=0)
        self.params['bdot_rate_1'] = parameter('bdot_rate',idx=1)
        self.params['bdot_dmag_0'] =  parameter('bdot_dmag',idx=0)
        self.params['bdot_dmag_1'] =  parameter('bdot_dmag',idx=1)
        self.params['bdot_dmag_2'] =  parameter('bdot_dmag',idx=2)       
        self.params['bdot_detumb'] =  parameter('bdot_detumb')       
              
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions) 

In [248]:
class SPEQS:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}

        self.params['en_payload'] = parameter('en_payload')               
        self.params['file_no'] = parameter('file_no')
        self.params['exp_profile'] = parameter('exp_profile')
        self.params['force_start'] = parameter('force_start')               
        self.params['exp_delay_s'] = parameter('exp_delay_s')  
        self.params['en_pheating'] = parameter('en_pheating')               
        self.params['pheat_tout'] = parameter('pheat_tout')               
        self.params['heater_setpt'] = parameter('heater_setpt')               
        self.params['fixed_LCPR'] = parameter('fixed_LCPR')               
        self.params['LD_current'] = parameter('LD_current')               
        self.params['exp_time'] = parameter('exp_time')               
        self.params['LCPR_ref'] = parameter('LCPR_ref')               
        self.params['LCPR_step'] = parameter('LCPR_step')               
        self.params['loop_counter'] = parameter('loop_counter')               
        self.params['btr'] = parameter('btr')                             
        self.params['APD1_offset'] = parameter('APD1_offset')                             
        self.params['APD2_offset'] = parameter('APD2_offset')                             
        self.params['LCPR1_buf_0'] =  parameter('LCPR1_Buf',idx=0)
        self.params['LCPR1_buf_1'] =  parameter('LCPR1_Buf',idx=1)
        self.params['LCPR1_buf_2'] =  parameter('LCPR1_Buf',idx=2)                            
        self.params['LCPR1_buf_3'] =  parameter('LCPR1_Buf',idx=3) 
        self.params['LCPR2_buf_0'] =  parameter('LCPR2_Buf',idx=0)
        self.params['LCPR2_buf_1'] =  parameter('LCPR2_Buf',idx=1)
        self.params['LCPR2_buf_2'] =  parameter('LCPR2_Buf',idx=2)                            
        self.params['LCPR2_buf_3'] =  parameter('LCPR2_Buf',idx=3)         
        
        
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions)  

In [249]:
class QRNG:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}

        self.params['active'] = parameter('active')               
        self.params['paramUpdIntval'] = parameter('paramUpdIntval')               
        self.params['curSending'] = parameter('curSending')               
        self.params['curQueueCnt'] = parameter('curQueueCnt')               
        self.params['curFilePos'] = parameter('curFilePos')               
        self.params['fileTimeStart'] = parameter('fileTimeStart')               
        self.params['fileReachEnd'] = parameter('fileReachEnd')               
        self.params['curFile'] = parameter('curFile')               
        self.params['curTime'] = parameter('curTime') 

        for index in range(32):
            self.params['prevData_'+ str(index)] = parameter('prevData',idx=index)
            self.params['curData_'+ str(index)] = parameter('curData',idx=index)

    #    self.params['LCPR1_buf_0'] =  parameter('LCPR1_Buf',idx=0 
        
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions)  

In [250]:
eps = EPS()
obc = OBC()
ax100 = AX100()
adcs = ADCS()
speqs = SPEQS()
qrng = QRNG()

eps.reload()
obc.reload()
ax100.reload()
adcs.reload()
speqs.reload()
qrng.reload()

In [251]:
for pname,param in qrng.params.items():
    print (pname, "\t:", end='')
    param.status()

prevData_30 	:prevData idx= 30 node= None len(Ts)= 0 len(Vals)= 0
prevData_9 	:prevData idx= 9 node= None len(Ts)= 0 len(Vals)= 0
currData_28 	:currData idx= 28 node= None len(Ts)= 0 len(Vals)= 0
currData_25 	:currData idx= 25 node= None len(Ts)= 0 len(Vals)= 0
prevData_4 	:prevData idx= 4 node= None len(Ts)= 0 len(Vals)= 0
paramUpdIntval 	:paramUpdIntval idx= None node= None len(Ts)= 0 len(Vals)= 0
prevData_16 	:prevData idx= 16 node= None len(Ts)= 0 len(Vals)= 0
currData_29 	:currData idx= 29 node= None len(Ts)= 0 len(Vals)= 0
prevData_25 	:prevData idx= 25 node= None len(Ts)= 0 len(Vals)= 0
prevData_27 	:prevData idx= 27 node= None len(Ts)= 0 len(Vals)= 0
currData_27 	:currData idx= 27 node= None len(Ts)= 0 len(Vals)= 0
currData_31 	:currData idx= 31 node= None len(Ts)= 0 len(Vals)= 0
prevData_3 	:prevData idx= 3 node= None len(Ts)= 0 len(Vals)= 0
prevData_31 	:prevData idx= 31 node= None len(Ts)= 0 len(Vals)= 0
prevData_1 	:prevData idx= 1 node= None len(Ts)= 0 len(Vals)= 0
curFile

In [252]:
print(eps.params['cntWdtCsp_0'].Vals[0:20] )

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [253]:
#qrng = QRNG()

In [254]:
#qrng.reload()